#Descarga de Billboard Chart

In [ ]:
pip install billboard.py

In [ ]:
#Librerías que se utilizarán
import pandas as pd
import requests
import json
import numpy as np
import billboard #API para descargar billboard charts
from datetime import date
from datetime import datetime
from datetime import timedelta

In [ ]:
fecha_inicial = datetime(2021, 6, 5) 
#Se crea una lista que almacena las fechas a extraer
m = 83 #Cantidad de semanas del ranking por extraer
fecha = fecha_inicial
dates = []
for i in range(0,m):
  dates.append(fecha)
  fecha = fecha + timedelta(days=7)

In [ ]:
#Se convierte la lista en una serie para obtener los datos en formato string
fechas = pd.Series(data = dates)
fechas = fechas.astype(str)

In [ ]:
#Se crea un dataframe vacío para almacenar los charts
hot100 = pd.DataFrame(columns = ["Ranking", "Canción", "Artista", "Ranking semana anterior", "Semanas en top100", "Fecha", "Máximo Ranking"], index = np.arange(0,100))
h100 = pd.DataFrame(columns = ["Ranking", "Canción", "Artista", "Ranking semana anterior", "Semanas en top100", "Fecha", "Máximo Ranking"])

In [ ]:
for i in range(0,m):
  chart = billboard.ChartData("hot-100", date = fechas.iloc[i]) #El argumento de date tiene que ser un string
  for k in range(0, 100):
    song = chart[k]
    hot100.iloc[k,1] = song.title
    hot100.iloc[k,5] = fechas.iloc[i]
    hot100.iloc[k,2] = song.artist
    hot100.iloc[k,0] = song.rank
    hot100.iloc[k,3] = song.lastPos
    hot100.iloc[k,6] = song.peakPos
    hot100.iloc[k,4] = song.weeks
  h100 = pd.concat([h100,hot100]) 

In [ ]:
#Se exporta el dataframe obtenido
h100.to_excel(r"Downloads\h100.xlsx")

# Descarga de atributos musicales de spotify

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [74]:
h100 = pd.read_excel('/content/drive/MyDrive/Curso Data Science/Desafío 8/Dataset seleccionado/Downloads_h100.xlsx')

In [75]:
pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
import spotipy
import os 
from spotipy.oauth2 import SpotifyClientCredentials
cid = '2422e7b85ead40f6a01a0bee5d5aa101'
secret = 'e583c32b186f4867a9863a9cfcc7190c'
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [56]:
from spotipy.oauth2 import SpotifyClientCredentials
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=cid, client_secret=secret))

In [84]:
tracks = h100.drop(columns = ["Ranking", "Semanas en top100", "Máximo Ranking", "Ranking semana anterior", "Unnamed: 0", "Fecha", "Canción ID", "Canción ID.1"])

In [85]:
tracks.insert(2,"Track_ID",0)

In [104]:
for i in range(7656,len(tracks)):
  artist = tracks.iloc[i,1]
  title = tracks.iloc[i,0]
  #results = spotify.search(q='artist:' + artist + ' track:' + title, type='track')
  results = sp.search(q=f"{tracks['Canción'][i]} {tracks['Artista'][i]} ", limit=5, type='track')
  track = pd.DataFrame.from_dict(results, orient = "index")
  #me da el uri, también podrìa ser con el ID
  track_id = results['tracks']['items'][0]['uri']
  tracks.iloc[i,2] = track_id

In [105]:
tracks

,Canción,Artista,Track_ID
0,Butter,BTS,spotify:track:6jjYDGxVJsWS0a5wlVF5vS
1,Good 4 U,Olivia Rodrigo,spotify:track:4ZtFanR9U6ndgddUvNcjcG
2,Deja Vu,Olivia Rodrigo,spotify:track:6HU7h9RYOaPRFeh0R3UeAr
3,Levitating,Dua Lipa,spotify:track:5nujrmhLynf4yMoMtj8AQF
4,Leave The Door Open,Silk Sonic (Bruno Mars & Anderson .Paak),spotify:track:02VBYrHfVwfEWXk5DXyf0T
...,...,...,...
8495,Down Home,Jimmie Allen,spotify:track:1EfTaVUbZmrtqniZTj6LYF
8496,One Thing At A Time,Morgan Wallen,spotify:track:7K3BhSpAxZBznislvUMVtn
8497,Hold Me Closer,Elton John & Britney Spears,spotify:track:72yP0DUlWPyH8P7IoxskwN
8498,Far,SZA,spotify:track:74NI58MQexwZjNu1Gu6GjT


In [ ]:
Audio = pd.DataFrame(columns = ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo", "s","r","f","fd","t","vdx","e"])
for i in range (0,len(tracks)):
  if tracks.iloc[i,2] != 0:
    s = sp.audio_features(tracks.iloc[i,2])
    son = pd.DataFrame(s)
  else:
    s = 0
  Audio = pd.concat([Audio,son])
